<a href="https://colab.research.google.com/github/chw8207/fastai_pytorch/blob/main/%ED%98%91%EC%97%85%ED%95%84%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow as tf
from fastai.collab import *
from fastai.tabular.all import *
import pandas as pd
import numpy as np

### GPU 설정

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0' :
    raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

Found GPU at: /device:GPU:0


In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7232764106261803215
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 40129593344
locality {
  bus_id: 1
  links {
  }
}
incarnation: 532635173440550084
physical_device_desc: "device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0"
xla_global_id: 416903419
]


### 데이터 : 무비렌즈

In [5]:
path = untar_data(URLs.ML_100k)

In [7]:
# 해당 테이블을 열고 내용을 살펴보기
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user','movie','rating','timestap'])
ratings.head()

,user,movie,rating,timestap
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [9]:
# SF, 액션, 고전 영화를 나타내는 요소(스타워즈 최신작)
rise_skywalker = np.array([0.98,0.9,-0.9])

In [10]:
# 현대적인 SF영화를 좋아하는 사용자
user1 = np.array([0.9,0.8,-0.6])

In [11]:
# 이 둘의 조합에서 유사성 계산
(user1*rise_skywalker).sum()

2.1420000000000003

In [12]:
# 영화 카사블랑카
casablanca = np.array([-0.99,-0.3,0.8])

In [13]:
# 사용자 조합해서 유사성 계산
(user1*casablanca).sum()

-1.611

### DataLoaers만들기

In [14]:
movies = pd.read_csv(path/'u.item', delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie','title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
